In [1]:
import ast
import json
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
#import plotly
#plotly.offline.init_notebook_mode(connected=True)
#import plotly.offline as py
import plotly.plotly as py
import plotly.tools as tls
import string
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from src.data import read_files
from collections import Counter
import itertools
from textblob import TextBlob
from langdetect import detect

import string
from googletrans import Translator
#from google.api_core.protobuf_helpers import get_messages
#from __future__ import absolute_import
from google.cloud import translate
from langdetect import detect
translator = Translator()
import math

import sys
reload(sys)
sys.setdefaultencoding('utf-8')

import csv

In [159]:
#hotel = pd.DataFrame.from_csv('./data/hotel_train.csv')
hotel = pd.read_csv('./data/hotel_phrase.csv')
foreign = pd.DataFrame.from_csv('./data/foreign.csv')

In [172]:
hotel['words_clean'] = hotel['text_clean'].apply(lambda x: str(x).split(' '))
hotel['words_clean'][foreign.index] = foreign['clean_words']

In [160]:
hotel['text_clean'][foreign.index] = foreign['text_clean']
hotel['phrases'][foreign.index] = foreign['phrases']

In [176]:
def count_fre(stars, ch):
    """
    count the frequency of 'ch': could be phrases or words
    corresponding to the 'stars': could be 1, 2, 3, 4, 5
    """
    p_list = list(itertools.chain(*list(hotel.loc[hotel['stars']==stars, ch])))
    p_count = pd.DataFrame(Counter(p_list).most_common())
    p = pd.DataFrame(p_count[p_count.columns[1]])
    p.index = p_count[p_count.columns[0]]
    p.columns = [str(stars)]
    return p

def str_to_list(s):
    """
    transform a string to a list, e.g. "[a,b,c]" to [a,b,c] 
    """
    s = s.replace("u'", "")
    s = s.replace("'", "")
    l = s[1:-1].split(',')
    return l

In [162]:
#counts of phrases frequency
hotel['phrases'] = hotel['phrases'].apply(lambda x: str_to_list(x))
p = pd.concat([count_fre(1, 'phrases'), count_fre(2,'phrases'),count_fre(3,'phrases'),count_fre(4,'phrases'),count_fre(5,'phrases')],axis=1)
p = p.sort_values(by=['1'], ascending=False)
p = p.fillna(0)

In [177]:
#counts of words frequency
w = pd.concat([count_fre(1, 'words_clean'), count_fre(2,'words_clean'),count_fre(3,'words_clean'),count_fre(4,'words_clean'),
               count_fre(5,'words_clean')],axis=1)
w = w.sort_values(by=['1'], ascending=False)
w = w.fillna(0)

In [175]:
hotel.to_csv('./data/hotel.csv',index=False)

In [145]:
def average(x):
    assert len(x) > 0
    return float(sum(x)) / len(x)

def pearson_def(x,y):
    """
    calculate the pearson correlation coefficient between two list: x & y
    """
    assert len(x) == len(y)
    n = len(x)
    assert n > 0
    avg_x = average(x)
    avg_y = average(y)
    diffprod = 0
    xdiff2 = 0
    ydiff2 = 0
    for idx in range(n):
        xdiff = x[idx] - avg_x
        ydiff = y[idx] - avg_y
        diffprod += xdiff * ydiff
        xdiff2 += xdiff * xdiff
        ydiff2 += ydiff * ydiff

    return diffprod / math.sqrt(xdiff2 * ydiff2)

def get_corr(row):
    x = [row['1'],row['2'],row['3'],row['4'],row['5']]
    return pearson_def(x,range(1,6))

In [166]:
p['correlation'] = p.apply(get_corr,axis=1)

In [167]:
p.to_csv('./data/hotel_phrase_count.csv')

In [168]:
p[(p['correlation']<-0.85) & (p['1']>85)].sort_values(by=['correlation'])

,1,2,3,4,5,correlation
didnt work,130.0,66.0,49.0,22.0,11.0,-0.950341
room didnt,125.0,74.0,72.0,58.0,26.0,-0.945755
hot water,341.0,194.0,159.0,132.0,68.0,-0.944952
room wasnt,255.0,149.0,136.0,121.0,56.0,-0.936746
room number,106.0,69.0,33.0,37.0,20.0,-0.930160
minutes check,96.0,50.0,44.0,30.0,20.0,-0.928442
resort fees,180.0,119.0,101.0,105.0,55.0,-0.928060
air conditioner,93.0,50.0,28.0,28.0,12.0,-0.925009
switch rooms,105.0,50.0,25.0,22.0,9.0,-0.912651
minutes line,101.0,46.0,28.0,18.0,13.0,-0.902726


In [178]:
p[(p['correlation']>0.85) & (p['5']>200)].sort_values(by=['correlation'], ascending=False)

,1,2,3,4,5,correlation
friendly staff,52.0,94.0,237.0,633.0,735.0,0.957997
happy hour,41.0,62.0,100.0,243.0,295.0,0.957473
beautiful hotel,38.0,55.0,95.0,241.0,330.0,0.953172
friendly helpful,71.0,126.0,217.0,605.0,856.0,0.951592
nice helpful,43.0,54.0,76.0,151.0,233.0,0.943100
clean staff,23.0,37.0,116.0,260.0,247.0,0.942687
great place,38.0,37.0,117.0,365.0,431.0,0.940631
great price,32.0,28.0,77.0,245.0,309.0,0.936720
prime rib,83.0,109.0,158.0,237.0,212.0,0.931628
full kitchen,26.0,32.0,91.0,297.0,307.0,0.930229


In [183]:
w['correlation'] = w.apply(get_corr,axis=1)

In [184]:
w.to_csv('./data/hotel_words_count.csv')

In [186]:
w[(w['correlation']<-0.85) & (w['1']>1000)].sort_values(by=['correlation'])

,1,2,3,4,5,correlation
disappointing,1138.0,985.0,709.0,456.0,135.0,-0.993755
uncomfortable,1221.0,814.0,522.0,384.0,155.0,-0.982223
carpet,2086.0,1160.0,859.0,585.0,356.0,-0.948057
gross,1332.0,655.0,428.0,327.0,101.0,-0.936773
hallway,1156.0,874.0,748.0,788.0,476.0,-0.934006
walls,1690.0,1175.0,1100.0,1066.0,581.0,-0.933242
elsewhere,1874.0,943.0,772.0,503.0,318.0,-0.929085
barely,1216.0,705.0,542.0,478.0,316.0,-0.928903
stains,1693.0,761.0,417.0,202.0,64.0,-0.926848
Unfortunately,1146.0,776.0,744.0,730.0,419.0,-0.918598


In [192]:
w[(w['correlation']>0.85) & (w['5']>5000)].sort_values(by=['correlation'], ascending=False)

,1,2,3,4,5,correlation
friendly,2717.0,3097.0,5439.0,11849.0,16019.0,0.953575
easy,867.0,977.0,2412.0,5647.0,6191.0,0.951123
comfortable,1196.0,1740.0,3761.0,8510.0,8518.0,0.948187
great,5940.0,6404.0,11878.0,31735.0,42094.0,0.942329
restaurants,1299.0,1734.0,3596.0,7354.0,6952.0,0.940592
fun,1530.0,1406.0,2742.0,6612.0,8982.0,0.940381
super,1716.0,1453.0,2578.0,5251.0,7029.0,0.937192
loved,419.0,451.0,1054.0,4156.0,5993.0,0.929643
Great,656.0,634.0,1594.0,5789.0,8863.0,0.928731
excellent,420.0,556.0,1108.0,3307.0,5744.0,0.928043
